In [ ]:
pip install google-auth google-api-python-client apscheduler requests

In [ ]:
import os
import re
import json
import time
import requests
import smtplib
from email.mime.text import MIMEText
from datetime import datetime, timedelta
from apscheduler.schedulers.background import BackgroundScheduler
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Configuration and Constants
SPREADSHEET_ID = '##########################'  #  Google Sheet ID
SHEET_NAME = "Form Responses"  # Updated sheet name 
RANGE_NAME = "!A1:E10"
API_KEY = '#############################'           # API key
WEBHOOK_URL = '#######################'
CANDIDATE_EMAIL = '####################'  # for the webhook header

# SMTP email settings 
SMTP_SERVER = os.environ.get('SMTP_SERVER', 'smtp.gmail.com')
SMTP_PORT = int(os.environ.get('SMTP_PORT', 587))
SMTP_USERNAME = os.environ.get('SMTP_USERNAME', '#####################')
SMTP_PASSWORD = os.environ.get('SMTP_PASSWORD', '###########')


# Google API Setup Functions using API Key
def get_google_services():
    sheets_service = build('sheets', 'v4', developerKey=API_KEY)
    drive_service = build('drive', 'v3', developerKey=API_KEY)
    return sheets_service, drive_service

def get_sheet_data(sheets_service):
    try:
        sheet = sheets_service.spreadsheets()
        result = sheet.values().get(
            spreadsheetId=SPREADSHEET_ID, 
            range=RANGE_NAME
        ).execute()
        values = result.get('values', [])
        return values
    except HttpError as err:
        print("Error fetching sheet data:", err)
        raise

def update_sheet_cell(sheets_service, row, col, value):
    cell = f'{chr(65 + col)}{row}'
    body = {'values': [[value]]}
    try:
        sheets_service.spreadsheets().values().update(
            spreadsheetId=SPREADSHEET_ID, 
            range=f'{SHEET_NAME}!{cell}',
            valueInputOption='RAW', 
            body=body
        ).execute()
    except HttpError as err:
        print(f"Error updating cell {cell}: {err}")


# Utility Functions
def extract_file_id(cv_link):
    """
    Extract the file ID from a Google Drive share link.
    Supports multiple URL formats:
      - https://drive.google.com/open?id=FILE_ID
      - https://drive.google.com/file/d/FILE_ID/view
      - https://drive.google.com/uc?id=FILE_ID
    """
    patterns = [
        r"drive\.google\.com/open\?id=([\w-]+)",
        r"drive\.google\.com/file/d/([\w-]+)/view",
        r"drive\.google\.com/uc\?id=([\w-]+)"
    ]
    
    for pattern in patterns:
        match = re.search(pattern, cv_link)
        if match:
            return match.group(1)
    return None  

def process_cv(file_id, drive_service):
    """
    Downloads the file from Drive and extracts key sections.
    In this demo, we return dummy data. For real parsing,
    integrate a PDF/DOCX parsing library or an external API.
    """
    try:
        request = drive_service.files().get_media(fileId=file_id)
        file_data = request.execute()
        extracted_data = {
            "education": ["Bachelor of Science in Computer Science"],
            "qualifications": ["Skilled in Python and API integration"],
            "projects": ["Job Application Pipeline Automation"]
        }
        return extracted_data
    except HttpError as err:
        print(f"Error processing CV file {file_id}: {err}")
        return {"education": [], "qualifications": [], "projects": []}

def send_webhook(payload):
    headers = {
        'Content-Type': 'application/json',
        'X-Candidate-Email': CANDIDATE_EMAIL
    }
    try:
        response = requests.post(WEBHOOK_URL, headers=headers, data=json.dumps(payload))
        print("Webhook response:", response.text)
    except Exception as e:
        print("Error sending webhook:", e)

def send_followup_email(recipient_email, recipient_name):
    subject = "Your CV is Under Review"
    body = (f"Hi {recipient_name},\n\n"
            "Thank you for your application. Your CV is currently under review. "
            "We will update you with further information shortly.\n\n"
            "Best regards,\nMetana Team")
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = SMTP_USERNAME
    msg['To'] = recipient_email
    try:
        server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
        server.starttls()
        server.login(SMTP_USERNAME, SMTP_PASSWORD)
        server.send_message(msg)
        server.quit()
        print(f"Follow-up email sent to {recipient_email}")
    except Exception as e:
        print(f"Error sending email to {recipient_email}: {e}")

def schedule_email(recipient_email, recipient_name, send_time):
    """
    Schedule the follow-up email using APScheduler.
    Note: The script must keep running for the scheduler to trigger jobs.
    """
    scheduler = BackgroundScheduler()
    scheduler.add_job(send_followup_email, 'date', run_date=send_time,
                      args=[recipient_email, recipient_name])
    scheduler.start()
    print(f"Scheduled follow-up email to {recipient_email} at {send_time}")



# Submission Processing
def process_submission(row, row_index, sheets_service, drive_service):
    """
    Processes a single form submission.
    Expected row format: [Timestamp, Name, Email, Phone Number, CV Document Upload]
    """
    if len(row) < 5:
        print(f"Row {row_index} is incomplete, skipping.")
        return

    name = row[1]
    email = row[2]
    cv_link = row[4]

    print(f"Processing submission from {name} ({email})")
    file_id = extract_file_id(cv_link)
    if not file_id:
        print(f"Could not extract file ID from CV link: {cv_link}")
        return

    cv_data = process_cv(file_id, drive_service)

    # Build payload
    payload = {
        "cv_data": {
            "personal_info": {
                "name": name,
                "email": email
            },
            "education": cv_data["education"],
            "qualifications": cv_data["qualifications"],
            "projects": cv_data["projects"],
            "cv_public_link": cv_link
        },
        "metadata": {
            "applicant_name": name,
            "email": email,
            "status": "prod", 
            "cv_processed": True,
            "processed_timestamp": datetime.utcnow().isoformat() + "Z"
        }
    }

    send_webhook(payload)

    # Schedule follow-up email for the next day
    now = datetime.utcnow()
    send_time = (now + timedelta(days=1)).replace(hour=9, minute=0, second=0, microsecond=0)
    schedule_email(email, name, send_time)

    print(f"Row {row_index} processed.")


# Execution
def main():
    sheets_service, drive_service = get_google_services()
    data = get_sheet_data(sheets_service)
    if not data:
        print("No data found in sheet.")
        return

    for i, row in enumerate(data[1:], start=2):
        process_submission(row, i, sheets_service, drive_service)

    print("Processing complete. The scheduler will run pending follow-up emails.")
    try:
        while True:
            time.sleep(60)
    except KeyboardInterrupt:
        print("Script terminated by user.")

if __name__ == "__main__":
    main()
